In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
import pickle

: 

In [ ]:
df = pd.read_excel('final.xlsx')
df.head()

: 

In [ ]:
df.shape

: 

In [ ]:
df.info()

: 

In [ ]:
df.columns

: 

In [ ]:
df.describe()

: 

In [ ]:
# REMOVING OUTLIERS
for i in df['id'].unique():
    temp_df = df[df['id']==i]
    avg_id = temp_df['total_sales_price'].mean()
    std_id = temp_df['total_sales_price'].std()
    for j in temp_df['total_sales_price']:
        if j < (avg_id - (3 * std_id)) or j > (avg_id + (3 * std_id)):
            df.drop( df[df['total_sales_price'] == j].index,axis=0,inplace=True)

: 

In [ ]:
df.shape

: 

In [ ]:
clarity_order = { 0 : 'IF', 1 : 'VVS1', 2 : 'VVS2', 3 : 'VS1', 4 : 'VS2', 5 : 'SI1', 6 : 'SI2', 7 : 'I1', 8 : 'I2', 9 : 'I3'}
cut_order = {0 : 'Excellent', 1 : 'Very Good'}
symmetry_order = {0 : 'Excellent', 1 : 'Very Good'}
polish_order = {0 : 'Excellent', 1 : 'Very Good'}
fluorescence_order = {0 : 'None', 1 : 'Faint', 2 : 'Medium', 3 : 'Strong', 4 : 'Very Strong'}
# girdle_max_order = {0 : 'XT', 1 : 'VTN', 2 : 'VTN', 3 : 'XTN', 4 : 'M',5 : 'TK',6 : 'STK',7 : 'VTK',8:'XTK'}
# girdle_min_order = {0 : 'TN', 1 : 'STN', 2 : 'VTN', 3 : 'XTN', 4 : 'M',5 : 'TK',6 : 'STK',7 : 'VTK',8:'XTK'}
culet_size_order = {0 : 'N', 1 : 'VS', 2 : 'S', 3 : 'M'}

: 

In [ ]:
df['clarity'].replace(to_replace=clarity_order.values(),value=clarity_order.keys(),inplace=True)
df['cut'].replace(to_replace=cut_order.values(),value=cut_order.keys(),inplace=True)
df['symmetry'].replace(to_replace=symmetry_order.values(),value=symmetry_order.keys(),inplace=True)
df['polish'].replace(to_replace=polish_order.values(),value=polish_order.keys(),inplace=True)
df['fluor_intensity'].replace(to_replace=fluorescence_order.values(),value=fluorescence_order.keys(),inplace=True)
# df['girdle_max'].replace(to_replace=girdle_max_order.values(),value=girdle_max_order.keys(),inplace=True)
# df['girdle_min'].replace(to_replace=girdle_min_order.values(),value=girdle_min_order.keys(),inplace=True)
df['culet_size'].replace(to_replace=culet_size_order.values(),value=culet_size_order.keys(),inplace=True)
df.head()

: 

In [ ]:
df.drop(['range','id'],axis=1,inplace=True)

: 

In [ ]:
df['ratio'] = round(df['ratio'],2)

: 

In [ ]:
df.head()

: 

In [ ]:
lb = LabelEncoder()
df['color'] = lb.fit_transform(df['color'])
df['girdle_min'] = lb.fit_transform(df['girdle_min'])
df['girdle_max'] = lb.fit_transform(df['girdle_max'])

: 

In [ ]:
x = df.drop(['total_sales_price'],axis=1)
y = df['total_sales_price']

: 

In [ ]:
plt.figure(figsize=(15,12))
vmin = -1
center = 0
vmax = 1
mask = np.triu(np.ones_like(pd.concat([x,y],axis=1).corr()))
  
# plotting the heatmap
hm = sns.heatmap(pd.concat([x,y],axis=1).corr(),vmin=vmin,vmax=vmax,center=center, annot=True,cmap="YlGnBu",mask=mask)

: 

: 

### FOR POINTERS

In [ ]:
ptr_df = df[df['size']<1]

: 

In [ ]:
ptr_df.head()

: 

In [ ]:
ptr_df.shape

: 

In [ ]:
x = ptr_df.drop(['total_sales_price'],axis=1)
y = ptr_df['total_sales_price']

: 

In [ ]:
plt.figure(figsize=(15,12))
vmin = -1
center = 0
vmax = 1
mask = np.triu(np.ones_like(pd.concat([x,y],axis=1).corr()))
  
# plotting the heatmap
hm = sns.heatmap(pd.concat([x,y],axis=1).corr(),vmin=vmin,vmax=vmax,center=center, annot=True,cmap="YlGnBu",mask=mask)

: 

In [ ]:
y=np.array(y)

: 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.75,random_state=123)

: 

In [ ]:
n_estimators = [410]
max_features = ['auto']
max_depth = [8]
min_samples_split = [4]
min_samples_leaf = [2]
bootstrap = [True]

: 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features,
    'max_depth' : max_depth,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf' : min_samples_leaf,
    'bootstrap' : bootstrap
}

: 

In [ ]:
model = RandomForestRegressor()

: 

In [ ]:
ptr_model = RandomizedSearchCV(estimator = model,param_distributions=random_grid,n_iter=15,cv=3,verbose=2)

: 

In [ ]:
ptr_model.fit(x_train,y_train)

: 

In [ ]:
ptr_model.best_params_

: 

In [ ]:
y_pred = ptr_model.predict(x_test)
y_pred

: 

In [ ]:
print(f'Train Accuracy :{ptr_model.score(x_train,y_train):.3f}')
print(f'Test Accuracy :{ptr_model.score(x_test,y_test):.3f}')

: 

In [ ]:
mean_absolute_error(y_test,y_pred)

: 

In [ ]:
result1_df = pd.DataFrame({'Predicted Price': y_pred,'Actual Price':y_test})
result1_df.head(10) 

: 

In [ ]:
result1_df["Error % using RF"] = ((result1_df["Predicted Price"] - result1_df["Actual Price"]) / result1_df["Actual Price"]) *100
result1_df = result1_df.round(2)
result1_df.head(10)

: 

In [ ]:
data = {
        'Model': ['Random Forest Regression for Pointers'],
        'Mean' : np.round([result1_df['Error % using RF'].mean()],decimals=2),
        'Max'  : [result1_df['Error % using RF'].max()],
        'Min'  : [result1_df['Error % using RF'].min()],
        'Variance': np.round([result1_df['Error % using RF'].var()],decimals=2)
        }

model_stats = pd.DataFrame(data)
model_stats

: 

: 

### FOR ONE-UPs

In [ ]:
one_up_df = df[df['size'] >= 1]
one_up_df.head()

: 

In [ ]:
one_up_df = one_up_df[one_up_df['size'] < 5]
one_up_df.head()

: 

In [ ]:
one_up_df.shape

: 

In [ ]:
x = one_up_df.drop(['total_sales_price'],axis=1)
y = one_up_df['total_sales_price']

: 

In [ ]:
plt.figure(figsize=(15,12))
vmin = -1
center = 0
vmax = 1
mask = np.triu(np.ones_like(pd.concat([x,y],axis=1).corr()))
  
# plotting the heatmap
hm = sns.heatmap(pd.concat([x,y],axis=1).corr(),vmin=vmin,vmax=vmax,center=center, annot=True,cmap="YlGnBu",mask=mask)

: 

In [ ]:
y=np.array(y)

: 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.75,random_state=123)

: 

In [ ]:
n_estimators = [550]
max_features = ['auto']
max_depth = [9]
min_samples_split = [5]
min_samples_leaf = [2]
bootstrap = [True]

: 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features,
    'max_depth' : max_depth,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf' : min_samples_leaf,
    'bootstrap' : bootstrap
}

: 

In [ ]:
model = RandomForestRegressor()

: 

In [ ]:
one_up_model = RandomizedSearchCV(estimator = model,param_distributions=random_grid,n_iter=22,cv=5,verbose=2)

: 

In [ ]:
one_up_model.fit(x_train,y_train)

: 

In [ ]:
one_up_model.best_params_

: 

In [ ]:
y_pred = one_up_model.predict(x_test)
y_pred

: 

In [ ]:
print(f'Train Accuracy :{one_up_model.score(x_train,y_train):.3f}')
print(f'Test Accuracy :{one_up_model.score(x_test,y_test):.3f}')

: 

In [ ]:
mean_absolute_error(y_test,y_pred)

: 

In [ ]:
result2_df = pd.DataFrame({'Predicted Price': y_pred,'Actual Price':y_test})
result2_df.head(10) 

: 

In [ ]:
result2_df["Error % using RF"] = ((result2_df["Predicted Price"] - result2_df["Actual Price"]) / result2_df["Actual Price"]) *100
result2_df = result2_df.round(2)
result2_df.head(10)

: 

In [ ]:
data = {
        'Model': ['Random Forest Regression for One-Ups'],
        'Mean' : np.round([result2_df['Error % using RF'].mean()],decimals=2),
        'Max'  : [result2_df['Error % using RF'].max()],
        'Min'  : [result2_df['Error % using RF'].min()],
        'Variance': np.round([result2_df['Error % using RF'].var()],decimals=2)
        }

model_stats = pd.DataFrame(data)
model_stats

: 

In [ ]:
model1 = open('ptr_model1.pkl','wb')
pickle.dump(ptr_model,model1)
model1.close()

: 

In [ ]:
model2 = open('one_up_model1.pkl','wb')
pickle.dump(one_up_model,model2)
model2.close()

: 

: 